In [1]:
import numpy as np
import pandas as pd
from glob import glob

import datetime as dts
from datetime import date
from datetime import timedelta

In [2]:
files = glob("Incoming Raw/*.csv")

In [3]:
df = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814426 entries, 0 to 814425
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   No                       814426 non-null  int64  
 1   No. Waybill              814426 non-null  object 
 2   No. Bagging              318112 non-null  object 
 3   No. Vehicle Tag          261999 non-null  object 
 4   Kota Incoming            814426 non-null  object 
 5   Tipe Scan                814426 non-null  object 
 6   Lokasi Sebelumnya        80245 non-null   object 
 7   Lokasi Selanjutnya       51938 non-null   object 
 8   Scan Tujuan              814426 non-null  object 
 9   Discan oleh              814426 non-null  object 
 10  Diinput Cabang           814426 non-null  object 
 11  Waktu Pencatatan         814426 non-null  object 
 12  Waktu Operasi            814426 non-null  object 
 13  Kurir                    388990 non-null  object 
 14  Caba

In [5]:
df.loc[0]

No                                           1
No. Waybill                    IDS901306496120
No. Bagging                                NaN
No. Vehicle Tag                   VA2200233794
Kota Incoming                HULU SUNGAI UTARA
Tipe Scan                         Sending scan
Lokasi Sebelumnya                          NaN
Lokasi Selanjutnya           TH AMUNTAI TENGAH
Scan Tujuan                      AMUNTAI UTARA
Discan oleh                     Muhammad Zaini
Diinput Cabang                      TH BARABAI
Waktu Pencatatan           01-02-2023 23:59:48
Waktu Operasi              01-02-2023 23:59:43
Kurir                                      NaN
Cabang Incoming                            NaN
Total paket                                  1
Berat                                     0.02
Berat aktual                              0.02
Volume Berat                               0.0
Jenis paket                             Barang
Jenis layanan                         Drop off
Giliran      

In [6]:
df["Waktu Pencatatan"] = pd.to_datetime(df["Waktu Pencatatan"], dayfirst=True)

In [7]:
df["Waktu Operasi"] = pd.to_datetime(df["Waktu Operasi"], dayfirst=True)

In [8]:
#df["Tanggal Pencatatan"] = df["Waktu Pencatatan"].dt.date

In [9]:
pickup = df[df["Tipe Scan"] == "Pick up scan"].copy()

In [10]:
delivery = df[df["Tipe Scan"].isin(["Delivery scan", "Delivery Task Accept"])].sort_values(by = ["Tipe Scan", "Waktu Pencatatan"], ascending=[True, True]).copy()

In [11]:
delivery.drop_duplicates(subset ="No. Waybill", inplace = True)

In [12]:
arrival = df[df["Tipe Scan"].isin(["Arrival scan", "Unloading scan", "Unpacking scan"])].copy()

In [13]:
arrival["Tipe Scan"].unique()

array(['Arrival scan', 'Unloading scan', 'Unpacking scan'], dtype=object)

In [14]:
pickup = pickup.merge(delivery[["No. Waybill", "Diinput Cabang", "Waktu Pencatatan"]].drop_duplicates(subset="No. Waybill"), 
                      how="left", left_on = "No. Waybill", right_on = "No. Waybill")

In [15]:
pickup = pickup[~pickup["Diinput Cabang_y"].isnull()].copy()

In [16]:
arrival = arrival.merge(delivery[["No. Waybill", "Diinput Cabang", "Waktu Pencatatan"]], 
                      how="left", left_on = "No. Waybill", right_on = "No. Waybill")

In [17]:
incoming = pd.concat([arrival, pickup]).copy()

In [18]:
scan = pd.CategoricalDtype(categories=['Arrival scan', 'Unloading scan', 'Unpacking scan', 'Pick up scan'], ordered=True)

In [19]:
incoming["Tipe Scan"] = incoming["Tipe Scan"].astype(scan)

In [20]:
incoming.rename(columns = {"Diinput Cabang_x":"Diinput Cabang", "Waktu Pencatatan_x":"Waktu Pencatatan"}, inplace = True)

In [21]:
incoming.rename(columns = {"Diinput Cabang_y":"TH Delivery", "Waktu Pencatatan_y":"Waktu Delivery"}, inplace = True)

In [22]:
#incoming.sort_values(by = ["Tanggal Pencatatan", "Tipe Scan"], ascending = [False,True], inplace=True)

In [23]:
incoming.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263503 entries, 0 to 48036
Data columns (total 30 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   No                       263503 non-null  int64         
 1   No. Waybill              263503 non-null  object        
 2   No. Bagging              198855 non-null  object        
 3   No. Vehicle Tag          156907 non-null  object        
 4   Kota Incoming            263503 non-null  object        
 5   Tipe Scan                263503 non-null  category      
 6   Lokasi Sebelumnya        80245 non-null   object        
 7   Lokasi Selanjutnya       0 non-null       object        
 8   Scan Tujuan              263503 non-null  object        
 9   Discan oleh              263503 non-null  object        
 10  Diinput Cabang           263503 non-null  object        
 11  Waktu Pencatatan         263503 non-null  datetime64[ns]
 12  Waktu Operasi    

In [24]:
def codstatus(x):
    if x == 0:
        return "Non COD"
    else:
        return "COD"

In [25]:
incoming["COD Status"] = incoming["COD"].apply(lambda x: codstatus(x))

In [26]:
incoming["lookup"] = incoming["Kota Tujuan"] + incoming["Tujuan Waybill"] 

In [27]:
lookup = pd.read_excel("Lookup.xlsx")

In [28]:
incoming = incoming.merge(lookup[["Lookup", "Destinasi Area"]], 
                      how="left", left_on = "lookup", right_on = "Lookup")

In [29]:
incoming.drop(["lookup", "Lookup"], axis=1, inplace=True)

In [30]:
def tharrival(row):
    if row["Tipe Scan"] == "Pick up scan":
        return row["TH Delivery"]
    else:
        return row["Diinput Cabang"]

In [31]:
incoming["TH Arrival"] = incoming[['Tipe Scan','TH Delivery','Diinput Cabang']].apply(tharrival, axis=1)

In [32]:
def waktuarrival(row):
    if row["Tipe Scan"] == "Pick up scan":
        return row["Waktu Delivery"]
    else:
        return row["Waktu Pencatatan"]

In [33]:
incoming["Waktu Arrival"] = incoming[['Tipe Scan','Waktu Delivery','Waktu Pencatatan']].apply(waktuarrival, axis=1)

In [34]:
incoming["Tanggal Arrival"] = incoming["Waktu Arrival"].dt.date

In [35]:
incoming.sort_values(by = ["Tanggal Arrival","Tipe Scan","Waktu Arrival"], ascending = [True,True,False], inplace=True)

In [36]:
incoming.drop_duplicates(subset = ["No. Waybill", "Tanggal Arrival"], inplace = True)

In [37]:
incoming = incoming.merge(lookup[["Destinasi Area", "Mitra Area"]].drop_duplicates(subset="Destinasi Area"), 
                      how="left", left_on = "TH Arrival", right_on = "Destinasi Area")

In [38]:
incoming.rename(columns = {"Destinasi Area_x":"Destinasi Area"}, inplace = True)

In [39]:
incoming.drop(columns=["Destinasi Area_y"], inplace=True)

In [40]:
incoming["Waktu Arrival"] = pd.to_datetime(incoming["Waktu Arrival"], dayfirst=True)

In [41]:
incoming["Tanggal Arrival"] = pd.to_datetime(incoming["Tanggal Arrival"], dayfirst=True)

In [42]:
incoming.drop_duplicates(subset = ["No. Waybill", "Tanggal Arrival"], inplace = True)

In [43]:
incoming.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94686 entries, 0 to 94685
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   No                       94686 non-null  int64         
 1   No. Waybill              94686 non-null  object        
 2   No. Bagging              68053 non-null  object        
 3   No. Vehicle Tag          79687 non-null  object        
 4   Kota Incoming            94686 non-null  object        
 5   Tipe Scan                94686 non-null  category      
 6   Lokasi Sebelumnya        78878 non-null  object        
 7   Lokasi Selanjutnya       0 non-null      object        
 8   Scan Tujuan              94686 non-null  object        
 9   Discan oleh              94686 non-null  object        
 10  Diinput Cabang           94686 non-null  object        
 11  Waktu Pencatatan         94686 non-null  datetime64[ns]
 12  Waktu Operasi            94686 n

In [44]:
incoming.set_index("No", inplace=True)

In [45]:
incoming.sort_values(by = ["Tanggal Arrival", "Waktu Arrival"], ascending = [True,False], inplace=True)

In [46]:
def transit(row):
    if row["Tipe Scan"] == "Pick up scan":
        return np.nan
    elif row["TH Delivery"] == None and row["Destinasi Area"] == None:
        return "Transit"
    elif row["Destinasi Area"] == row["TH Arrival"]:
        return np.nan
    elif row["TH Delivery"] == row["TH Arrival"]:
        return np.nan
    elif row["TH Delivery"] == row["Destinasi Area"] and pd.Timestamp(row["Waktu Delivery"]).date() == pd.Timestamp(row["Tanggal Arrival"]).date():
        return np.nan
    else:
        return "Transit"
    

In [47]:
incoming["Transit"] = incoming.apply(lambda row: transit(row), axis=1)

In [48]:
incoming.isnull().sum()

No. Waybill                    0
No. Bagging                26633
No. Vehicle Tag            14999
Kota Incoming                  0
Tipe Scan                      0
Lokasi Sebelumnya          15808
Lokasi Selanjutnya         94686
Scan Tujuan                    0
Discan oleh                    0
Diinput Cabang                 0
Waktu Pencatatan               0
Waktu Operasi                  0
Kurir                      80397
Cabang Incoming            15808
Total paket                    0
Berat                          0
Berat aktual                   0
Volume Berat                   0
Jenis paket                    0
Jenis layanan                  0
Giliran                    94686
Deskripsi                  94686
Tujuan Waybill                 0
Kota Tujuan                    0
Provinsi Tujuan Waybill        0
PAD                            0
COD                            0
TH Delivery                 3112
Waktu Delivery              3112
COD Status                     0
Destinasi 

In [49]:
bulan = (date.today() - timedelta(days=1)).month
bulan

2

In [50]:
incoming = incoming[incoming["Tanggal Arrival"].dt.month == bulan]

In [51]:
bersih = incoming[incoming["Transit"].isnull()].sort_values(by=["Tanggal Arrival", "Waktu Arrival"], ascending=[True, True]).drop_duplicates(subset="No. Waybill").copy()

In [52]:
bersih.sort_values(by = ["Tanggal Arrival", "Tipe Scan", "Waktu Pencatatan"], ascending=[True, True, False], inplace=True)

In [53]:
kemaren = (date.today() - timedelta(days=1)).strftime('%d %b %Y')
kemaren

'15 Feb 2023'

In [54]:
bersih.to_csv(f"incoming {kemaren}.csv", sep=",", index=False)

In [55]:
def deliverytime(row):
    if row["Waktu Delivery"] == None:
        return np.nan
    elif row["Waktu Delivery"] > row["Waktu Arrival"]:
        deliv = (row["Waktu Delivery"] - row["Waktu Arrival"])
        return deliv
    else:
        return np.nan


In [56]:
#bersih["Delivery Time"] = bersih[["Waktu Delivery", "Waktu Arrival"]].apply(deliverytime, axis=1)

In [57]:
#left = pd.DataFrame({'key': ['A', 'B', 'C', 'D'], 'value': np.random.randn(4)})
#right = pd.DataFrame({'key': ['B', 'B', 'B', 'E', 'F'], 'value': np.random.randn(5)})
#left.merge(right, how="left", left_on='key', right_on='key')

In [58]:
df[df["Tipe Scan"] == "Confirm Return Bill"].value_counts(subset="Provinsi Tujuan Waybill")

Provinsi Tujuan Waybill
KALIMANTAN SELATAN      95
KALIMANTAN BARAT        40
KALIMANTAN TIMUR        31
KALIMANTAN TENGAH       23
KALIMANTAN UTARA         9
DKI JAKARTA              5
JAWA BARAT               3
SULAWESI SELATAN         2
SUMATERA SELATAN         2
JAWA TIMUR               2
BANTEN                   1
KEP. BANGKA BELITUNG     1
KEP. RIAU                1
LAMPUNG                  1
NTT                      1
SULAWESI TENGGARA        1
SUMATERA BARAT           1
BALI                     1
dtype: int64